# 1. Install Requirements

In [1]:
# YOLOv7 and requirements
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

fatal: destination path 'yolov7' already exists and is not an empty directory.
/Users/antonios/Desktop/Dissertation/Model/Scuba_Diving_Project/yolov7
  Using cached matplotlib-3.6.1-cp310-cp310-macosx_11_0_arm64.whl (7.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 40.8 MB/s eta 0:00:0000:0100:01
  Using cached opencv_python-4.6.0.66-cp37-abi3-macosx_11_0_arm64.whl (30.0 MB)
  Using cached Pillow-9.2.0-cp310-cp310-macosx_11_0_arm64.whl (2.8 MB)
  Using cached PyYAML-6.0-cp310-cp310-macosx_11_0_arm64.whl (173 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached scipy-1.9.2-cp310-cp310-macosx_12_0_arm64.whl (28.5 MB)
  Using cached torch-1.12.1-cp310-none-macosx_11_0_arm64.whl (49.1 MB)
  Using cached torchvision-0.13.1-cp310-cp310-macosx_11_0_arm64.whl (1.2 MB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached protobuf-4.21.2-cp37-abi3-macosx_10_9_universal2.whl (483 kB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 

  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached oauthlib-3.2.1-py3-none-any.whl (151 kB)


# 2. Download Annotated Dataset

In [ ]:
# Copy from Roboflow
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="CCByhgZhC987VAUFU6rg")
project = rf.workspace("computer-project").project("scuba-diving-world")
dataset = project.version(1).download("yolov7")

In [ ]:
# Training Command
%cd /content/yolov7
!python train.py --batch 16 --epochs 50 --data {dataset.location}/data.yaml --weights 'yolov7_training.pt' --device 0 

# 3. Model Training

In [ ]:
# COCO starting checkpoint
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
# Training Code
%cd /content/yolov7
!python train.py --batch 16 --epochs 30 --data {dataset.location}/data.yaml --weights 'yolov7_training.pt' --device 0 


# 4. Export


In [ ]:
#  zip to download weights and results locally

!zip -r export.zip runs/detect
!zip -r export.zip runs/train/exp/weights/best.pt
!zip export.zip runs/train/exp/*

# 5. Convert YOLOv7 weight to TFLite for Android App

In [ ]:
!pip --quiet install onnx onnxruntime onnxsim
!pip install onnx-tf

In [ ]:
%cd /content/yolov7
!python export.py --weights best.pt --grid --end2end --simplify \
--topk-all 100 --iou-thres 0.65 --conf-thres 0.35 --img-size 640 640 --max-wh 640

In [ ]:
!onnx-tf convert -i /content/yolov7/best.onnx -o /content/

In [ ]:
import cv2
import time
import requests
import random
import numpy as np
import onnxruntime as ort
from PIL import Image
from pathlib import Path
from collections import OrderedDict,namedtuple
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/')
tflite_model = converter.convert()

with open('/content/yolov7_model.tflite', 'wb') as f:
  f.write(tflite_model)# Colab - Convert YOLOv7 weight to TFLite for Android App

# 6. Download the TFLite